# micrograd

In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def f(x):
    return 3*x**2 - 4*x + 5

In [ ]:
f(3.0)

In [ ]:
xs = np.arange(-5, 5)
ys = f(xs)
plt.plot(xs, ys)

The derivative of a function ***f(x)*** is the amount ***f(x)*** changes when a small value ***h*** is added to ***x***, as ***h*** approaches `0`.
$$ \lim_{h \to 0} \frac{f(x+h) - f(x)}{h}$$

In [ ]:
h = 0.000001
x = 2/3
(f(x+h) - f(x)) / h # Dividing by h 'normalizes' the difference

In [ ]:
# More complex function d(a, b, c) = a * b + c
h = 0.0001

# Inputs
a = 2.0
b =  -3.0
c = 10.0

d1 = a * b + c
a += h
d2 = a * b + c

print("d1:", d1)
print("d2:", d2)
print("Slope:", (d2-d1)/h)
# Note: da/dd = b = -3.0
# Note: db/dd = a = 2.0
# Note: dc/dd = so the slope will be 1

In [8]:
class Value:
    def __init__(self, data, _children=(), _op="") -> None:
        self.data = data
        self._prev = set(_children)
        self._op = _op

    def __repr__(self) -> str:
        return f"Value(data={self.data})"
    
    def __add__(self, other):
        out = Value(self.data + other.data, _children=(self, other), _op="+")
        return out

    def __mul__(self, other):
        out = Value(self.data * other.data, _children=(self, other), _op="*")
        return out

a = Value(2.0)
b = Value(-3.0)
c = Value(10.0)
d = a * b + c

In [11]:
from graphviz import Digraph

def trace(root):
    """Builds a set of all nodes and edges in the graph"""
    nodes, edges = set(), set()
    def build(v):
        if v not in nodes:
            nodes.add(v)
            for child in v._prev:
                edges.add((child, v))
                build(child)
    build(root)
    return nodes, edges

def draw_dot(root):
    dot = Digraph(format="svg", graph_attr={"rankdir": "LR"}) # LR means Left-to-Right
    
    nodes, edges = trace(root)
    for node in nodes:
        uid = str(id(node))
        # For any value in the graph create a rectangular ('record') node for it
        dot.node(name=uid, label=f"data: {node.data:.4f}")
        if node._op:
            # If this value is the result of an operation, create an op node for it
            dot.node(name = uid + node._op, label = node._op)
            # and connect this node to it.
            dot.edge(uid + node._op, uid)
    
    for parent, child in edges:
        # Connect the parent to the op node of it's child
        dot.edge(str(id(parent)), str(id(child)) + child._op)
    return dot

In [13]:
draw_dot(d)